In [12]:
import numpy as np
import pandas as pd

import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from pathlib import Path

from copy import deepcopy

data_path = Path('')
train_path = data_path / 'arc-agi_training_challenges.json'
train_sols_path = data_path / 'arc-agi_training_solutions.json'
eval_path = data_path / 'arc-agi_evaluation_challenges.json'
eval_sols_path = data_path / 'arc-agi_evaluation_solutions.json'

In [19]:
with open(train_path, 'r') as f:
    train_tasks = json.load(f)
with open(train_sols_path, 'r') as f:
    train_sols = json.load(f)
with open(eval_path, 'r') as f:
    eval_tasks = json.load(f)
with open(eval_sols_path, 'r') as f:
    eval_sols = json.load(f)

In [57]:
# Rotates a grid 90 degrees right
def rotate(grid):
    return [list(reversed(col)) for col in zip(*grid)]

# Mirrors a grid over a vertical line
def mirror(grid):
    return [list(reversed(row)) for row in grid]

# Eight dihedral symmetries
transformations = ["", "r", "rr", "rrr", "m", "mr", "mrr", "mrrr"]

def apply_transformation(grid, transformation):
    for op in transformation:
        if op == "r":
            grid = rotate(grid)
        if op == "m":
            grid = mirror(grid)
    return grid

In [58]:
# Augments tasks and solutions using the eight dihedral symmetries
def dihedral_augment(tasks, sols):
    augmented_tasks = {}
    augmented_sols = {}
    
    for key in tasks:
        for transformation in transformations:
            new_key = key + "_" + transformation
            new_task = deepcopy(tasks[key])
            new_sol = deepcopy(sols[key])
            num_train = len(new_task['train'])
            num_test = len(new_task['test'])
            for i in range(num_train):
                new_task['train'][i]['input'] = apply_transformation(new_task['train'][i]['input'], transformation)
                new_task['train'][i]['output'] = apply_transformation(new_task['train'][i]['output'], transformation)
            for i in range(num_test):
                new_task['test'][i]['input'] = apply_transformation(new_task['test'][i]['input'], transformation)
                new_sol[i] = apply_transformation(new_sol[i], transformation)
            augmented_tasks[new_key] = new_task
            augmented_sols[new_key] = new_sol
            
    return augmented_tasks, augmented_sols

In [59]:
augmented_train_tasks, augmented_train_sols = dihedral_augment(train_tasks, train_sols)

In [61]:
with open('dihedral_augmented_arc-agi_training_challenges.json', 'w') as f:
    json.dump(augmented_train_tasks, f)

In [62]:
with open('dihedral_augmented_arc-agi_training_solutions.json', 'w') as f:
    json.dump(augmented_train_sols, f)